<a href="https://colab.research.google.com/github/amy0621/LLM/blob/main/shit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# 安裝相依套件
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install fastapi uvicorn librosa onnx onnxruntime

# 下載 DeepVoiceGuard 專案
!git clone https://github.com/Mrkomiljon/DeepVoiceGuard.git
%cd DeepVoiceGuard


Looking in indexes: https://download.pytorch.org/whl/cu118
Cloning into 'DeepVoiceGuard'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 81 (delta 26), reused 0 (delta 0), pack-reused 30 (from 2)
Receiving objects: 100% (81/81), 127.86 MiB | 27.11 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (23/23), done.
/content/DeepVoiceGuard/DeepVoiceGuard/DeepVoiceGuard/DeepVoiceGuard/DeepVoiceGuard


In [55]:
from google.colab import files

print("⬆️ 請上傳音檔（例如 spoof_example.wav）")
audio_file = files.upload()


⬆️ 請上傳音檔（例如 spoof_example.wav）


KeyboardInterrupt: 

In [57]:
import os
import torch
import yaml
import librosa
import numpy as np
from model import RawNet  # 確保你有 model/__init__.py 和 model/RawNet.py
from google.colab import files # Import files here if not already imported in this cell

# 檢查使用哪個裝置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1️⃣ 設定模型與音檔路徑
# 尋找上傳的 .pth 模型檔
model_path = "checkpoints/best_model.pth"

# 尋找上傳的音檔 ( assuming .wav extension)
audio_path = None
for filename in audio_file.keys():
    if filename.endswith('.wav') or filename.endswith('.mp3'): # Added .mp3 based on audio_path variable
        audio_path = filename
        break

if not model_path or not audio_path:
    raise FileNotFoundError("模型檔 (.pth) 或音檔 (.wav/.mp3) 未上傳喔公主 👑")


# 2️⃣ 載入模型設定
with open('model_config_RawNet.yaml', 'r') as f:
    config = yaml.safe_load(f)['model']
model = RawNet(config, device).to(device)

# 3️⃣ 修正 PyTorch 2.6 的 bug，加入 weights_only=False 👇
# model_path 現在應該指向正確的 .pth 檔案
model_loaded = torch.load(model_path, map_location=device, weights_only=False)
if isinstance(model_loaded, dict) and 'state_dict' in model_loaded:
    model.load_state_dict(model_loaded['state_dict'])  # 若是 checkpoint 格式
else:
    model.load_state_dict(model_loaded)  # 若是純 state_dict
model.eval()

# 4️⃣ 載入音檔
audio, sr = librosa.load(audio_path, sr=16000)
expected_len = 64600
if len(audio) > expected_len:
    audio = audio[:expected_len]
else:
    audio = np.pad(audio, (0, expected_len - len(audio)))

audio_tensor = torch.FloatTensor(audio).unsqueeze(0).to(device)

# 5️⃣ 推論！
with torch.no_grad():
    output = model(audio_tensor)
    # Assuming the second element (index 1) is the 'spoof' class
    prediction = torch.sigmoid(output)[:, 1].item() # Select the second element and convert to scalar

# 6️⃣ 輸出結果
print(" 這段語音是詐騙 " if prediction < 0.5 else " 這段語音是正常語音")

 這段語音是詐騙 


# 新增區段